# BIP v10: Extracting the Mathematical Structure of Ethics

**If the model works, what IS the structure it learned?**

This notebook analyzes the trained z_bond latent space to discover:

1. **Dimensionality**: How many dimensions does ethics really need?
2. **Geometry**: What shape is the moral space? (Simplex? Lattice? Manifold?)
3. **Algebra**: Are there group operations? (obligation + permission = ?)
4. **Topology**: Are there clusters? Holes? Connected components?
5. **Symmetries**: What transformations preserve moral structure?

---

## Candidate Mathematical Structures

| Structure | Description | Fits if... |
|-----------|-------------|------------|
| **Lattice** | Partial ordering with meet/join | Moral concepts have clear hierarchy |
| **Simplex** | Convex hull of vertices | Concepts are "pure", blends are interior |
| **Lie Group** | Continuous symmetry group | Moral transformations are smooth |
| **Graph** | Nodes + edges | Discrete relationships dominate |
| **Fiber Bundle** | Base (universal) + fiber (cultural) | Universal core + cultural variation |
| **Boolean Algebra** | AND/OR/NOT operations | Moral logic is classical |
| **Deontic Logic** | O(p), P(p), F(p) operators | Obligation/Permission/Forbidden |

---

## Hohfeld's Framework (Already Algebraic!)

```
        RIGHT ←--correlative--→ DUTY
          ↑                       ↑
       opposite                opposite  
          ↓                       ↓
      NO-RIGHT ←--correlative--→ PRIVILEGE
```

This is a **square of opposition** - the same structure as Aristotelian logic!

---

In [ ]:
#@title 1. Setup & Load Trained Model { display-mode: "form" }
#@markdown **Two options:**
#@markdown 1. **Google Drive**: If you ran v9 with Drive mounted
#@markdown 2. **Upload**: If you downloaded the zip from v9

#@markdown ---
USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
#@markdown Set to True if you have results in Google Drive, False to upload zip file.

import os, subprocess, sys
for dep in ["transformers", "torch", "sentence-transformers", "scikit-learn", "umap-learn", "networkx", "scipy"]:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", dep])

import torch
import torch.nn as nn
import numpy as np
import json
from pathlib import Path
from collections import defaultdict
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import networkx as nx

try:
    import umap
    HAS_UMAP = True
except:
    HAS_UMAP = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Create directories
os.makedirs("models", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("data/splits", exist_ok=True)

if USE_GOOGLE_DRIVE:
    # Mount Drive
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_DIR = '/content/drive/MyDrive/BIP_native_v9'
    print(f"Looking for trained models in: {SAVE_DIR}")
    
    if os.path.exists(SAVE_DIR):
        print("\nAvailable files:")
        for f in os.listdir(SAVE_DIR):
            print(f"  {f}")
    else:
        print("WARNING: No trained models found in Drive. Run v9 first!")
        
else:
    # Upload zip file
    print("="*60)
    print("UPLOAD MODE")
    print("="*60)
    print()
    print("Please upload the 'bip_v9_complete.zip' file from v9.")
    print()
    
    from google.colab import files
    uploaded = files.upload()
    
    if uploaded:
        zip_name = list(uploaded.keys())[0]
        print(f"\nReceived: {zip_name}")
        
        # Extract
        import zipfile
        with zipfile.ZipFile(zip_name, 'r') as z:
            z.extractall('uploaded_data')
        
        print("Extracted contents:")
        !find uploaded_data -type f
        
        # Set paths
        SAVE_DIR = 'uploaded_data'
        
        # Copy to expected locations
        !cp uploaded_data/models/*.pt models/ 2>/dev/null || true
        !cp uploaded_data/data/*.jsonl data/processed/ 2>/dev/null || true
        !cp uploaded_data/data/*.json data/splits/ 2>/dev/null || true
        
        print("\nFiles ready for analysis!")
    else:
        print("ERROR: No file uploaded!")
        SAVE_DIR = None

print()
print("="*60)


In [ ]:
#@title 2. Define Model Architecture (must match training) { display-mode: "form" }

from transformers import AutoModel, AutoTokenizer

class GradientReversal(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.clone()
    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambda_ * grad_output, None

def gradient_reversal(x, lambda_=1.0):
    return GradientReversal.apply(x, lambda_)

class BIPModel(nn.Module):
    def __init__(self, d_model=384, d_bond=64, n_bonds=10, n_langs=5, n_periods=10, n_hohfeld=4):
        super().__init__()
        self.encoder = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        
        self.bond_proj = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(d_model // 2, d_bond)
        )
        
        self.bond_classifier = nn.Linear(d_bond, n_bonds)
        self.hohfeld_classifier = nn.Linear(d_bond, n_hohfeld)
        self.language_classifier = nn.Linear(d_bond, n_langs)
        self.period_classifier = nn.Linear(d_bond, n_periods)
    
    def extract_z_bond(self, input_ids, attention_mask):
        with torch.no_grad():
            out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
            h = (out.last_hidden_state * attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(-1, keepdim=True).clamp(min=1e-9)
            return self.bond_proj(h)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

print("Model architecture defined.")
print(f"z_bond dimension: 64")


In [ ]:
#@title 3. Extract z_bond Embeddings from All Data { display-mode: "form" }
#@markdown Extracts the learned moral embeddings for analysis.

from torch.utils.data import Dataset, DataLoader

# Load model
model = BIPModel().to(device)

# Try to load best model
# Try multiple locations (Drive, uploaded, local)
model_paths = [
    # Uploaded/extracted location
    "models/best_hebrew_to_others.pt",
    "models/best_mixed_baseline.pt",
    "models/best_semitic_to_non_semitic.pt",
    "models/best_ancient_to_modern.pt",
    # Drive location
    f"{SAVE_DIR}/best_hebrew_to_others.pt" if SAVE_DIR else "",
    f"{SAVE_DIR}/best_mixed_baseline.pt" if SAVE_DIR else "",
    f"{SAVE_DIR}/best_semitic_to_non_semitic.pt" if SAVE_DIR else "",
    # Uploaded subfolder
    "uploaded_data/models/best_hebrew_to_others.pt",
    "uploaded_data/models/best_mixed_baseline.pt",
]
model_paths = [p for p in model_paths if p]  # Remove empty strings

loaded = False
for path in model_paths:
    if os.path.exists(path):
        print(f"Loading model from {path}")
        model.load_state_dict(torch.load(path, map_location='cpu'))
        model = model.to(device)
        model.eval()
        loaded = True
        break

if not loaded:
    print("WARNING: No trained model found!")
    print("Using randomly initialized model for demonstration.")

# Load data
class SimpleDataset(Dataset):
    def __init__(self, passages_file, bonds_file, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.data = []
        
        with open(passages_file, 'r') as fp, open(bonds_file, 'r') as fb:
            for p_line, b_line in zip(fp, fb):
                p = json.loads(p_line)
                b = json.loads(b_line)
                if b['passage_id'] == p['id']:
                    self.data.append({
                        'text': p['text'][:500],
                        'language': p['language'],
                        'period': p['time_period'],
                        'bond': b['bonds']['primary_bond'],
                        'hohfeld': b['bonds'].get('hohfeld'),
                    })
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        enc = self.tokenizer(item['text'], truncation=True, max_length=self.max_len,
                            padding='max_length', return_tensors='pt')
        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'language': item['language'],
            'period': item['period'],
            'bond': item['bond'],
            'hohfeld': item['hohfeld'],
        }

def collate_fn(batch):
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'languages': [x['language'] for x in batch],
        'periods': [x['period'] for x in batch],
        'bonds': [x['bond'] for x in batch],
        'hohfelds': [x['hohfeld'] for x in batch],
    }

# Check for data
# Try multiple data locations
data_paths = [
    # Local (copied from upload)
    ("data/processed/passages.jsonl", "data/processed/bonds.jsonl"),
    # Uploaded subfolder
    ("uploaded_data/data/passages.jsonl", "uploaded_data/data/bonds.jsonl"),
    # Drive location
    (f"{SAVE_DIR}/processed/passages.jsonl", f"{SAVE_DIR}/processed/bonds.jsonl") if SAVE_DIR else ("", ""),
]
data_paths = [(p, b) for p, b in data_paths if p and b]

dataset = None
for p_path, b_path in data_paths:
    if os.path.exists(p_path) and os.path.exists(b_path):
        print(f"Loading data from {p_path}")
        dataset = SimpleDataset(p_path, b_path, tokenizer)
        print(f"Loaded {len(dataset):,} samples")
        break

if dataset is None:
    print("WARNING: No data found! Run v9 first.")
else:
    # Extract embeddings
    print("\nExtracting z_bond embeddings...")
    
    # Sample if too large
    max_samples = 50000
    if len(dataset) > max_samples:
        indices = np.random.choice(len(dataset), max_samples, replace=False)
        subset = torch.utils.data.Subset(dataset, indices)
    else:
        subset = dataset
    
    loader = DataLoader(subset, batch_size=256, shuffle=False, collate_fn=collate_fn, num_workers=2)
    
    all_z = []
    all_languages = []
    all_periods = []
    all_bonds = []
    all_hohfelds = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Extracting", unit="batch"):
            z = model.extract_z_bond(batch['input_ids'].to(device), batch['attention_mask'].to(device))
            all_z.append(z.cpu().numpy())
            all_languages.extend(batch['languages'])
            all_periods.extend(batch['periods'])
            all_bonds.extend(batch['bonds'])
            all_hohfelds.extend(batch['hohfelds'])
    
    Z = np.vstack(all_z)
    print(f"\nExtracted {Z.shape[0]:,} embeddings of dimension {Z.shape[1]}")
    
    # Summary
    print(f"\nLanguages: {set(all_languages)}")
    print(f"Bonds: {set(all_bonds)}")
    print(f"Periods: {len(set(all_periods))} unique")


In [ ]:
#@title 4. Analyze Dimensionality: How Many Dimensions Does Ethics Need? { display-mode: "form" }
#@markdown Uses PCA to find the intrinsic dimensionality of moral space.

print("="*60)
print("DIMENSIONALITY ANALYSIS")
print("="*60)
print()
print("Question: How many dimensions does moral cognition really need?")
print()

# PCA analysis
pca_full = PCA()
pca_full.fit(Z)

# Variance explained
var_explained = pca_full.explained_variance_ratio_
cum_var = np.cumsum(var_explained)

print("Variance explained by top components:")
for i in [1, 2, 3, 5, 10, 20, 32, 64]:
    if i <= len(cum_var):
        print(f"  {i:2d} dimensions: {cum_var[i-1]*100:5.1f}%")

# Find 90%, 95%, 99% thresholds
for thresh in [0.90, 0.95, 0.99]:
    n_dims = np.argmax(cum_var >= thresh) + 1
    print(f"\n{thresh*100:.0f}% variance requires {n_dims} dimensions")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(1, min(21, len(var_explained)+1)), var_explained[:20])
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Variance Explained')
axes[0].set_title('Scree Plot: Variance per Component')

axes[1].plot(range(1, len(cum_var)+1), cum_var, 'b-', linewidth=2)
axes[1].axhline(y=0.90, color='r', linestyle='--', label='90%')
axes[1].axhline(y=0.95, color='g', linestyle='--', label='95%')
axes[1].set_xlabel('Number of Dimensions')
axes[1].set_ylabel('Cumulative Variance')
axes[1].set_title('How Many Dimensions for Ethics?')
axes[1].legend()
axes[1].set_xlim([0, 64])

plt.tight_layout()
plt.savefig('dimensionality_analysis.png', dpi=150)
plt.show()

print()
print("INTERPRETATION:")
if cum_var[2] > 0.5:
    print("  → Ethics is LOW-DIMENSIONAL: First 3 components capture >50%")
    print("  → Suggests a simple geometric structure (e.g., simplex, plane)")
else:
    print("  → Ethics is HIGH-DIMENSIONAL: Need many components")
    print("  → Suggests complex manifold structure")


In [ ]:
#@title 5. Visualize Moral Space (2D Projections) { display-mode: "form" }
#@markdown Projects the 64D space to 2D for visualization.

print("="*60)
print("VISUALIZING MORAL SPACE")
print("="*60)

# Reduce to 2D using different methods
pca_2d = PCA(n_components=2)
Z_pca = pca_2d.fit_transform(Z)

print("Running t-SNE (this takes a minute)...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
Z_tsne = tsne.fit_transform(Z[:10000])  # Subsample for speed

if HAS_UMAP:
    print("Running UMAP...")
    reducer = umap.UMAP(n_components=2, random_state=42)
    Z_umap = reducer.fit_transform(Z[:10000])

# Color maps
bond_colors = {
    'HARM_PREVENTION': 'red',
    'RECIPROCITY': 'orange',
    'AUTONOMY': 'yellow',
    'PROPERTY': 'green',
    'FAMILY': 'blue',
    'AUTHORITY': 'purple',
    'CARE': 'pink',
    'FAIRNESS': 'cyan',
    'CONTRACT': 'brown',
    'NONE': 'gray',
}

lang_colors = {
    'hebrew': 'blue',
    'aramaic': 'lightblue',
    'classical_chinese': 'red',
    'arabic': 'green',
    'english': 'orange',
}

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# PCA by bond type
ax = axes[0, 0]
for bond in set(all_bonds):
    mask = [b == bond for b in all_bonds]
    ax.scatter(Z_pca[mask, 0], Z_pca[mask, 1], 
               c=bond_colors.get(bond, 'gray'), label=bond, alpha=0.3, s=10)
ax.set_title('PCA: Colored by Bond Type')
ax.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)')

# PCA by language
ax = axes[0, 1]
for lang in set(all_languages):
    mask = [l == lang for l in all_languages]
    ax.scatter(Z_pca[mask, 0], Z_pca[mask, 1],
               c=lang_colors.get(lang, 'gray'), label=lang, alpha=0.3, s=10)
ax.set_title('PCA: Colored by Language')
ax.legend(loc='upper right', fontsize=8)

# t-SNE by bond
ax = axes[1, 0]
for bond in set(all_bonds[:10000]):
    mask = [b == bond for b in all_bonds[:10000]]
    ax.scatter(Z_tsne[mask, 0], Z_tsne[mask, 1],
               c=bond_colors.get(bond, 'gray'), label=bond, alpha=0.3, s=10)
ax.set_title('t-SNE: Colored by Bond Type')

# t-SNE by language  
ax = axes[1, 1]
for lang in set(all_languages[:10000]):
    mask = [l == lang for l in all_languages[:10000]]
    ax.scatter(Z_tsne[mask, 0], Z_tsne[mask, 1],
               c=lang_colors.get(lang, 'gray'), label=lang, alpha=0.3, s=10)
ax.set_title('t-SNE: Colored by Language')

# UMAP if available
if HAS_UMAP:
    ax = axes[0, 2]
    for bond in set(all_bonds[:10000]):
        mask = [b == bond for b in all_bonds[:10000]]
        ax.scatter(Z_umap[mask, 0], Z_umap[mask, 1],
                   c=bond_colors.get(bond, 'gray'), label=bond, alpha=0.3, s=10)
    ax.set_title('UMAP: Colored by Bond Type')
    
    ax = axes[1, 2]
    for lang in set(all_languages[:10000]):
        mask = [l == lang for l in all_languages[:10000]]
        ax.scatter(Z_umap[mask, 0], Z_umap[mask, 1],
                   c=lang_colors.get(lang, 'gray'), label=lang, alpha=0.3, s=10)
    ax.set_title('UMAP: Colored by Language')
else:
    axes[0, 2].text(0.5, 0.5, 'UMAP not available', ha='center', va='center')
    axes[1, 2].text(0.5, 0.5, 'UMAP not available', ha='center', va='center')

plt.tight_layout()
plt.savefig('moral_space_visualization.png', dpi=150)
plt.show()

print()
print("INTERPRETATION:")
print("  - If bonds cluster but languages DON'T: Ethics is universal!")
print("  - If languages cluster: Model learned language, not ethics")
print("  - Mixed: Some universal structure, some cultural variation")


In [ ]:
#@title 6. Cluster Analysis: Are There Natural Moral Categories? { display-mode: "form" }
#@markdown Tests if the embedding space has natural clusters.

print("="*60)
print("CLUSTER ANALYSIS")
print("="*60)
print()
print("Question: Does moral space have discrete categories or continuous gradients?")
print()

# Try different numbers of clusters
silhouette_scores = []
k_range = range(2, 15)

print("Testing K-Means with different K...")
for k in tqdm(k_range):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(Z[:10000])
    score = silhouette_score(Z[:10000], labels)
    silhouette_scores.append(score)

best_k = k_range[np.argmax(silhouette_scores)]
print(f"\nBest K by silhouette: {best_k}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(list(k_range), silhouette_scores, 'b-o')
axes[0].axvline(x=best_k, color='r', linestyle='--', label=f'Best K={best_k}')
axes[0].set_xlabel('Number of Clusters')
axes[0].set_ylabel('Silhouette Score')
axes[0].set_title('How Many Natural Categories?')
axes[0].legend()

# Cluster with best K and visualize
kmeans_best = KMeans(n_clusters=best_k, random_state=42, n_init=10)
cluster_labels = kmeans_best.fit_predict(Z[:10000])

axes[1].scatter(Z_tsne[:, 0], Z_tsne[:, 1], c=cluster_labels, cmap='tab10', alpha=0.3, s=10)
axes[1].set_title(f'K-Means Clusters (K={best_k})')

plt.tight_layout()
plt.savefig('cluster_analysis.png', dpi=150)
plt.show()

# Analyze what's in each cluster
print(f"\nAnalyzing {best_k} clusters:")
for i in range(best_k):
    mask = cluster_labels == i
    cluster_bonds = [all_bonds[j] for j in range(len(mask)) if mask[j]]
    cluster_langs = [all_languages[j] for j in range(len(mask)) if mask[j]]
    
    bond_counts = defaultdict(int)
    lang_counts = defaultdict(int)
    for b in cluster_bonds:
        bond_counts[b] += 1
    for l in cluster_langs:
        lang_counts[l] += 1
    
    top_bond = max(bond_counts, key=bond_counts.get)
    top_lang = max(lang_counts, key=lang_counts.get)
    
    print(f"\n  Cluster {i} (n={sum(mask)}):")
    print(f"    Top bond: {top_bond} ({bond_counts[top_bond]/sum(mask)*100:.1f}%)")
    print(f"    Top lang: {top_lang} ({lang_counts[top_lang]/sum(mask)*100:.1f}%)")

print()
print("INTERPRETATION:")
if max(silhouette_scores) > 0.3:
    print(f"  → DISCRETE structure: Clear clusters (silhouette={max(silhouette_scores):.2f})")
else:
    print(f"  → CONTINUOUS structure: No clear clusters (silhouette={max(silhouette_scores):.2f})")


In [ ]:
#@title 7. Compute Bond Centroids: The "Platonic" Moral Concepts { display-mode: "form" }
#@markdown Finds the center of each bond type - the "ideal" moral concept.

print("="*60)
print("BOND CENTROIDS: Platonic Ideals of Moral Concepts")
print("="*60)
print()
print("Each centroid represents the 'pure' form of a moral concept.")
print()

# Compute centroids for each bond type
bond_centroids = {}
bond_stds = {}

for bond in set(all_bonds):
    mask = [b == bond for b in all_bonds]
    Z_bond = Z[mask]
    if len(Z_bond) > 10:
        bond_centroids[bond] = Z_bond.mean(axis=0)
        bond_stds[bond] = Z_bond.std(axis=0).mean()  # Average std across dimensions

print("Bond centroids computed:")
for bond, std in sorted(bond_stds.items(), key=lambda x: x[1]):
    n = sum(b == bond for b in all_bonds)
    print(f"  {bond:20s}: n={n:>6,}, spread={std:.3f}")

# Distance matrix between centroids
bonds = list(bond_centroids.keys())
n_bonds = len(bonds)
dist_matrix = np.zeros((n_bonds, n_bonds))

for i, b1 in enumerate(bonds):
    for j, b2 in enumerate(bonds):
        dist_matrix[i, j] = np.linalg.norm(bond_centroids[b1] - bond_centroids[b2])

# Visualize distance matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

im = axes[0].imshow(dist_matrix, cmap='viridis')
axes[0].set_xticks(range(n_bonds))
axes[0].set_yticks(range(n_bonds))
axes[0].set_xticklabels(bonds, rotation=45, ha='right')
axes[0].set_yticklabels(bonds)
axes[0].set_title('Distance Between Bond Centroids')
plt.colorbar(im, ax=axes[0])

# Project centroids to 2D
centroid_matrix = np.array([bond_centroids[b] for b in bonds])
pca_centroids = PCA(n_components=2)
centroids_2d = pca_centroids.fit_transform(centroid_matrix)

axes[1].scatter(centroids_2d[:, 0], centroids_2d[:, 1], s=200, c='red', zorder=5)
for i, bond in enumerate(bonds):
    axes[1].annotate(bond, (centroids_2d[i, 0], centroids_2d[i, 1]), fontsize=9, ha='center')
axes[1].set_title('Bond Centroids in 2D (PCA)')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')

plt.tight_layout()
plt.savefig('bond_centroids.png', dpi=150)
plt.show()

print()
print("NEAREST NEIGHBORS (which concepts are most similar):")
for i, b1 in enumerate(bonds):
    dists = [(bonds[j], dist_matrix[i, j]) for j in range(n_bonds) if i != j]
    dists.sort(key=lambda x: x[1])
    print(f"  {b1:20s} → nearest: {dists[0][0]} ({dists[0][1]:.3f})")


In [ ]:
#@title 8. Test for Algebraic Structure: Moral Arithmetic { display-mode: "form" }
#@markdown Tests if moral concepts have algebraic relationships.
#@markdown e.g., Does OBLIGATION - AUTHORITY + FAMILY = FILIAL_PIETY?

print("="*60)
print("ALGEBRAIC STRUCTURE: Moral Arithmetic")
print("="*60)
print()
print("Testing if moral concepts have vector arithmetic like word2vec.")
print("(king - man + woman = queen)")
print()

def find_nearest_bond(vec, exclude=None):
    """Find the bond whose centroid is nearest to vec."""
    min_dist = float('inf')
    nearest = None
    for bond, centroid in bond_centroids.items():
        if exclude and bond in exclude:
            continue
        dist = np.linalg.norm(vec - centroid)
        if dist < min_dist:
            min_dist = dist
            nearest = bond
    return nearest, min_dist

# Test some moral analogies
analogies = [
    # (A, B, C) -> A - B + C = ?
    ('AUTHORITY', 'HARM_PREVENTION', 'CARE'),  # Authority without harm plus care = ?
    ('FAMILY', 'PROPERTY', 'CARE'),  # Family without property plus care = ?
    ('CONTRACT', 'PROPERTY', 'FAMILY'),  # Contract without property plus family = ?
    ('RECIPROCITY', 'HARM_PREVENTION', 'CARE'),  # Reciprocity without harm plus care = ?
    ('FAIRNESS', 'AUTHORITY', 'AUTONOMY'),  # Fairness without authority plus autonomy = ?
]

print("Testing moral analogies (A - B + C = ?):")
print()

for A, B, C in analogies:
    if A in bond_centroids and B in bond_centroids and C in bond_centroids:
        # Compute A - B + C
        result_vec = bond_centroids[A] - bond_centroids[B] + bond_centroids[C]
        nearest, dist = find_nearest_bond(result_vec, exclude={A, B, C})
        
        print(f"  {A} - {B} + {C}")
        print(f"    = {nearest} (distance: {dist:.3f})")
        print()

# Test if there are consistent "directions" in moral space
print("\nSearching for moral DIRECTIONS (consistent transformations):")
print()

# Compute all pairwise differences
directions = {}
for b1 in bonds:
    for b2 in bonds:
        if b1 != b2:
            directions[(b1, b2)] = bond_centroids[b2] - bond_centroids[b1]

# Find directions that are similar across different pairs
print("Testing if some transformations are universal...")

# e.g., Is HARM→CARE similar to AUTHORITY→FAMILY?
test_pairs = [
    (('HARM_PREVENTION', 'CARE'), ('AUTHORITY', 'FAMILY')),
    (('PROPERTY', 'RECIPROCITY'), ('CONTRACT', 'FAIRNESS')),
    (('AUTONOMY', 'AUTHORITY'), ('CARE', 'HARM_PREVENTION')),
]

for (a1, a2), (b1, b2) in test_pairs:
    if (a1, a2) in directions and (b1, b2) in directions:
        d1 = directions[(a1, a2)]
        d2 = directions[(b1, b2)]
        
        # Cosine similarity
        cos_sim = np.dot(d1, d2) / (np.linalg.norm(d1) * np.linalg.norm(d2) + 1e-9)
        
        print(f"  {a1}→{a2} vs {b1}→{b2}")
        print(f"    Cosine similarity: {cos_sim:.3f}")
        if cos_sim > 0.5:
            print(f"    → These transformations are SIMILAR!")
        print()


In [ ]:
#@title 9. Build Moral Graph: Network Structure { display-mode: "form" }
#@markdown Builds a graph where edges connect similar moral concepts.

print("="*60)
print("MORAL GRAPH: Network Structure of Ethics")
print("="*60)
print()

# Build graph from bond centroids
G = nx.Graph()

# Add nodes
for bond in bonds:
    G.add_node(bond)

# Add edges based on distance (threshold)
threshold = np.percentile(dist_matrix[dist_matrix > 0], 30)  # Connect closest 30%

for i, b1 in enumerate(bonds):
    for j, b2 in enumerate(bonds):
        if i < j and dist_matrix[i, j] < threshold:
            G.add_edge(b1, b2, weight=1.0 / (dist_matrix[i, j] + 0.01))

print(f"Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Threshold distance: {threshold:.3f}")

# Graph metrics
print(f"\nGraph properties:")
print(f"  Connected: {nx.is_connected(G)}")
if nx.is_connected(G):
    print(f"  Diameter: {nx.diameter(G)}")
    print(f"  Avg clustering: {nx.average_clustering(G):.3f}")

# Centrality
betweenness = nx.betweenness_centrality(G)
print(f"\nMost central concepts (betweenness):")
for bond, centrality in sorted(betweenness.items(), key=lambda x: -x[1])[:5]:
    print(f"  {bond}: {centrality:.3f}")

# Visualize
fig, ax = plt.subplots(figsize=(12, 10))

pos = nx.spring_layout(G, k=2, iterations=50)
nx.draw_networkx_nodes(G, pos, node_size=2000, node_color='lightblue', ax=ax)
nx.draw_networkx_labels(G, pos, font_size=10, ax=ax)
nx.draw_networkx_edges(G, pos, alpha=0.5, ax=ax)

ax.set_title('Moral Concept Graph\n(edges connect similar concepts)')
ax.axis('off')

plt.tight_layout()
plt.savefig('moral_graph.png', dpi=150)
plt.show()

# Check for community structure
try:
    from networkx.algorithms import community
    communities = community.greedy_modularity_communities(G)
    print(f"\nDetected {len(communities)} communities:")
    for i, comm in enumerate(communities):
        print(f"  Community {i+1}: {', '.join(comm)}")
except:
    pass


In [ ]:
#@title 10. Test Lattice Structure (Hohfeld's Framework) { display-mode: "form" }
#@markdown Tests if the embedding space respects Hohfeld's deontic square.

print("="*60)
print("TESTING HOHFELD'S LATTICE STRUCTURE")
print("="*60)
print()
print("Hohfeld's framework says:")
print("  RIGHT ←correlative→ DUTY")
print("  RIGHT ←opposite→ NO_RIGHT")
print("  This forms a 'square of opposition'")
print()

# Compute Hohfeld centroids
hohfeld_centroids = {}
for h in ['OBLIGATION', 'RIGHT', 'LIBERTY', None]:
    mask = [hf == h for hf in all_hohfelds]
    if sum(mask) > 10:
        hohfeld_centroids[str(h)] = Z[mask].mean(axis=0)

print("Hohfeld centroids:")
for h, centroid in hohfeld_centroids.items():
    n = sum(str(hf) == h for hf in all_hohfelds)
    print(f"  {h}: n={n:,}")

if len(hohfeld_centroids) >= 3:
    # Compute distances
    hs = list(hohfeld_centroids.keys())
    print("\nDistances between Hohfeld states:")
    for i, h1 in enumerate(hs):
        for j, h2 in enumerate(hs):
            if i < j:
                d = np.linalg.norm(hohfeld_centroids[h1] - hohfeld_centroids[h2])
                print(f"  {h1} ↔ {h2}: {d:.3f}")
    
    # Test if opposites are far, correlatives are close
    print("\nTesting Hohfeld structure:")
    
    if 'OBLIGATION' in hohfeld_centroids and 'LIBERTY' in hohfeld_centroids:
        d_opp = np.linalg.norm(hohfeld_centroids['OBLIGATION'] - hohfeld_centroids['LIBERTY'])
        print(f"  OBLIGATION ↔ LIBERTY (should be opposite): {d_opp:.3f}")
    
    if 'OBLIGATION' in hohfeld_centroids and 'RIGHT' in hohfeld_centroids:
        d_corr = np.linalg.norm(hohfeld_centroids['OBLIGATION'] - hohfeld_centroids['RIGHT'])
        print(f"  OBLIGATION ↔ RIGHT (correlatives): {d_corr:.3f}")
    
    # Project to 2D
    if len(hohfeld_centroids) >= 2:
        H = np.array([hohfeld_centroids[h] for h in hs])
        pca_h = PCA(n_components=2)
        H_2d = pca_h.fit_transform(H)
        
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.scatter(H_2d[:, 0], H_2d[:, 1], s=500, c='red', zorder=5)
        for i, h in enumerate(hs):
            ax.annotate(h, (H_2d[i, 0], H_2d[i, 1]), fontsize=12, ha='center', va='bottom')
        
        # Draw expected square structure
        ax.set_title('Hohfeld States in Embedding Space\n(Do they form a square?)')
        ax.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
        ax.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('hohfeld_structure.png', dpi=150)
        plt.show()


In [ ]:
#@title 11. Extract Simplified Model: The Essence of Moral Structure { display-mode: "form" }
#@markdown Distills the learned structure into interpretable form.

print("="*60)
print("EXTRACTING THE ESSENCE: Simplified Moral Structure")
print("="*60)
print()

# 1. Key dimensions from PCA
print("1. PRINCIPAL MORAL DIMENSIONS")
print("-" * 40)

pca_full = PCA(n_components=10)
Z_reduced = pca_full.fit_transform(Z)

print("Top dimensions and what they capture:")
for i in range(min(5, len(pca_full.components_))):
    # Project bond centroids onto this dimension
    scores = {}
    for bond, centroid in bond_centroids.items():
        scores[bond] = np.dot(centroid, pca_full.components_[i])
    
    sorted_bonds = sorted(scores.items(), key=lambda x: x[1])
    low = sorted_bonds[:2]
    high = sorted_bonds[-2:]
    
    print(f"\n  PC{i+1} ({pca_full.explained_variance_ratio_[i]*100:.1f}% var):")
    print(f"    Low:  {low[0][0]} ({low[0][1]:.2f}), {low[1][0]} ({low[1][1]:.2f})")
    print(f"    High: {high[-1][0]} ({high[-1][1]:.2f}), {high[-2][0]} ({high[-2][1]:.2f})")

# 2. Nearest neighbor structure
print("\n\n2. MORAL NEAREST NEIGHBORS")
print("-" * 40)
print("Each concept's closest relative:")

nn_graph = {}
for i, b1 in enumerate(bonds):
    min_dist = float('inf')
    nearest = None
    for j, b2 in enumerate(bonds):
        if i != j and dist_matrix[i, j] < min_dist:
            min_dist = dist_matrix[i, j]
            nearest = b2
    nn_graph[b1] = nearest
    print(f"  {b1:20s} → {nearest}")

# 3. Hierarchical structure
print("\n\n3. HIERARCHICAL STRUCTURE")
print("-" * 40)

from scipy.cluster.hierarchy import linkage, dendrogram

centroid_matrix = np.array([bond_centroids[b] for b in bonds])
linkage_matrix = linkage(centroid_matrix, method='ward')

fig, ax = plt.subplots(figsize=(12, 6))
dendrogram(linkage_matrix, labels=bonds, ax=ax, leaf_rotation=45)
ax.set_title('Hierarchical Clustering of Moral Concepts')
ax.set_ylabel('Distance')
plt.tight_layout()
plt.savefig('moral_hierarchy.png', dpi=150)
plt.show()

# 4. Summary: The mathematical structure
print("\n\n" + "="*60)
print("SUMMARY: THE MATHEMATICAL STRUCTURE OF ETHICS")
print("="*60)

n_effective_dims = np.argmax(np.cumsum(pca_full.explained_variance_ratio_) > 0.90) + 1

print(f"""
Based on this analysis, ethics appears to have:

1. DIMENSIONALITY: ~{n_effective_dims} effective dimensions (90% variance)
   - Much lower than the 64D embedding space
   - Suggests ethics has a relatively simple structure

2. GEOMETRY: 
   - {'Discrete clusters' if max(silhouette_scores) > 0.3 else 'Continuous manifold'}
   - {best_k} natural categories identified

3. KEY RELATIONSHIPS:
   - Nearest neighbors reveal conceptual similarity
   - Some algebraic structure (moral arithmetic partially works)

4. HIERARCHY:
   - Concepts group into families
   - See dendrogram for structure

5. HOHFELD STRUCTURE:
   - {"Respects deontic logic" if len(hohfeld_centroids) >= 3 else "Insufficient data"}
""")

# Save the simplified model
simplified_model = {
    'bond_centroids': {k: v.tolist() for k, v in bond_centroids.items()},
    'pca_components': pca_full.components_[:n_effective_dims].tolist(),
    'pca_explained_variance': pca_full.explained_variance_ratio_[:n_effective_dims].tolist(),
    'nearest_neighbors': nn_graph,
    'n_effective_dimensions': int(n_effective_dims),
    'optimal_clusters': int(best_k),
    'distances': {f"{bonds[i]}_{bonds[j]}": float(dist_matrix[i,j]) 
                  for i in range(len(bonds)) for j in range(i+1, len(bonds))},
}

with open('simplified_moral_structure.json', 'w') as f:
    json.dump(simplified_model, f, indent=2)

print("\nSimplified model saved to 'simplified_moral_structure.json'")
print("This is the extracted mathematical structure of ethics!")


In [ ]:
#@title 12. Generate LaTeX for Paper { display-mode: "form" }
#@markdown Generates LaTeX tables and figures for publication.

print("="*60)
print("GENERATING LATEX FOR PUBLICATION")
print("="*60)

# Table 1: Bond centroids distances
print("\n% Table: Distance matrix between moral concepts")
print("\\begin{table}[h]")
print("\\centering")
print("\\caption{Euclidean distances between bond type centroids}")
print("\\begin{tabular}{l" + "c"*min(6, len(bonds)) + "}")
print("\\toprule")
print(" & " + " & ".join(bonds[:6]) + " \\\\")
print("\\midrule")
for i, b1 in enumerate(bonds[:6]):
    row = b1
    for j, b2 in enumerate(bonds[:6]):
        if i == j:
            row += " & -"
        else:
            row += f" & {dist_matrix[i,j]:.2f}"
    row += " \\\\"
    print(row)
print("\\bottomrule")
print("\\end{tabular}")
print("\\end{table}")

# Table 2: PCA results
print("\n% Table: Principal components of moral space")
print("\\begin{table}[h]")
print("\\centering")
print("\\caption{Principal dimensions of moral embedding space}")
print("\\begin{tabular}{ccc}")
print("\\toprule")
print("Component & Variance Explained & Cumulative \\\\")
print("\\midrule")
for i in range(min(10, len(pca_full.explained_variance_ratio_))):
    cum = sum(pca_full.explained_variance_ratio_[:i+1])
    print(f"PC{i+1} & {pca_full.explained_variance_ratio_[i]*100:.1f}\\% & {cum*100:.1f}\\% \\\\")
print("\\bottomrule")
print("\\end{tabular}")
print("\\end{table}")

print("\n" + "="*60)
print("LaTeX tables generated above.")
print("Copy-paste into your paper!")


In [ ]:
#@title 13. Save All Results { display-mode: "form" }

import shutil
from google.colab import files

!mkdir -p analysis_results

# Copy all figures
for f in ['dimensionality_analysis.png', 'moral_space_visualization.png', 
          'cluster_analysis.png', 'bond_centroids.png', 'moral_graph.png',
          'hohfeld_structure.png', 'moral_hierarchy.png', 'simplified_moral_structure.json']:
    if os.path.exists(f):
        !cp "{f}" analysis_results/

shutil.make_archive('moral_structure_analysis', 'zip', 'analysis_results')

print("Analysis results:")
!ls -la analysis_results/

files.download('moral_structure_analysis.zip')
